## EDA ##

In [ ]:
# Install dependencies
!pip install ultralytics -q

In [ ]:
# Import libraries
import zipfile
import os
import cv2
import matplotlib.pyplot as plt
from google.colab import files
from pathlib import Path
import pandas as pd
import numpy as np
from glob import glob
import json
from PIL import Image

In [ ]:
# Define supported image formats
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

In [ ]:
# Helper function to draw bounding boxes from YOLO txt
# This function takes an image and its corresponding YOLO-format label file
# and draws the bounding boxes with class names.
def draw_yolo_box(img_path, label_path, class_names):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            class_id, x_center, y_center, bw, bh = map(float, parts)
            xmin = int((x_center - bw / 2) * w)
            ymin = int((y_center - bh / 2) * h)
            xmax = int((x_center + bw / 2) * w)
            ymax = int((y_center + bh / 2) * h)
            label = class_names[int(class_id)] if class_names else str(int(class_id))
            cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)
            cv2.putText(img, label, (xmin, ymin - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
    return img

In [ ]:
## Analyze image metadata and data quality
# This function checks image dimensions, label availability, class counts, and highlights empty or missing labels.
def analyze_image_metadata(images_folder, labels_folder):
    image_meta = []
    class_distribution = {}
    empty_label_files = 0
    corrupted_images = 0

    for img_path in glob(os.path.join(images_folder, '*')):
        if not any(img_path.lower().endswith(ext) for ext in image_extensions):
            continue
        label_path = img_path.replace('/images/', '/labels/').rsplit('.', 1)[0] + '.txt'
        try:
            img = Image.open(img_path)
            width, height = img.size
        except:
            print(f'Corrupted image: {img_path}')
            corrupted_images += 1
            continue

        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                lines = f.readlines()
            num_objects = len(lines)
            if num_objects == 0:
                empty_label_files += 1
            for line in lines:
                class_id = line.strip().split()[0]
                class_distribution[class_id] = class_distribution.get(class_id, 0) + 1
        else:
            num_objects = 0
            empty_label_files += 1

        image_meta.append({
            'filename': os.path.basename(img_path),
            'width': width,
            'height': height,
            'num_objects': num_objects
        })

    df_meta = pd.DataFrame(image_meta)
    print("\nImage-level descriptive statistics:")
    print(df_meta.describe())

    print("\nEmpty or missing label files:", empty_label_files)
    print("Corrupted images:", corrupted_images)

    # Visualize object distribution per image
    plt.hist(df_meta['num_objects'], bins=range(0, df_meta['num_objects'].max() + 2), edgecolor='black')
    plt.title('Objects per Image')
    plt.xlabel('Number of Objects')
    plt.ylabel('Frequency')
    plt.show()

    # Plot class ID distribution
    if class_distribution:
        class_df = pd.DataFrame.from_dict(class_distribution, orient='index', columns=['Count'])
        class_df = class_df.sort_values(by='Count', ascending=False)
        print("\nClass distribution (raw class IDs):")
        print(class_df)
        class_df.plot(kind='bar', title='Class Distribution by ID')
        plt.xlabel("Class ID")
        plt.ylabel("Frequency")
        plt.show()

In [ ]:
# ========== POTHHOLE DATASET ==========
# Upload and analyze the pothole dataset
uploaded = files.upload()
zip_path = "/content/pothole.v18i.yolov8.zip"
extract_path = "/content/dataset/pothole"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted to:", extract_path)
analyze_image_metadata(f"{extract_path}/train/images", f"{extract_path}/train/labels")

KeyboardInterrupt: 

In [ ]:
# ========== FIRE DETECTION DATASET ==========
# Upload and analyze the fire detection dataset
uploaded = files.upload()
zip_path = "/content/fire detection.v1i.yolov8.zip"
extract_path = "/content/dataset/fire"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted to:", extract_path)
analyze_image_metadata(f"{extract_path}/train/images", f"{extract_path}/train/labels")

In [ ]:
# ========== FOG DETECTION DATASET ==========
# Upload and analyze the fog detection dataset
uploaded = files.upload()
zip_path = "/content/fog.v1i.yolov8.zip"
extract_path = "/content/dataset/fog"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted to:", extract_path)
analyze_image_metadata(f"{extract_path}/train/images", f"{extract_path}/train/labels")

In [ ]:
# ========== LANDSLIDE SEGMENTATION DATASET ==========
# Upload and analyze COCO-style landslide segmentation dataset
uploaded = files.upload()
zip_path = "/content/Segformer Landslide Detection.v4i.coco-segmentation.zip"
extract_path = "/content/dataset/landslide"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted to:", extract_path)

In [ ]:
# Locate annotation JSON
annotation_file = None
for root, dirs, files in os.walk(extract_path):
    for file in files:
        if file.endswith(".json"):
            annotation_file = os.path.join(root, file)
            break

In [ ]:
# Read and analyze COCO annotations
if annotation_file is None:
    print("No annotation JSON file found.")
else:
    with open(annotation_file, 'r') as f:
        coco_data = json.load(f)

    print("Total images:", len(coco_data['images']))
    print("Total annotations:", len(coco_data['annotations']))
    print("Total categories:", len(coco_data['categories']))

In [ ]:
# Print category ID to name mapping
    category_mapping = {cat['id']: cat['name'] for cat in coco_data['categories']}
    print("\nCategories:")
    for cid, name in category_mapping.items():
        print(f"  {cid}: {name}")

In [ ]:
# Count and display class distribution
    class_counts = {}
    for ann in coco_data['annotations']:
        cat_id = ann['category_id']
        name = category_mapping[cat_id]
        if name not in class_counts:
            class_counts[name] = 0
        class_counts[name] += 1

    df_class_counts = pd.DataFrame.from_dict(class_counts, orient='index', columns=['Count'])
    df_class_counts = df_class_counts.sort_values(by='Count', ascending=False)
    print("\nAnnotation distribution:")
    print(df_class_counts)

    df_class_counts.plot(kind='bar', figsize=(8, 4), legend=False, title='Segmentation Class Distribution - Landslide')
    plt.xlabel("Class")
    plt.ylabel("Count")
    plt.tight_layout()
    plt.show()

# Final confirmation
print("\nAll 4 datasets analyzed successfully. Full EDA including quality checks, statistics, and visualization is complete. Ready for training.")

IndentationError: unexpected indent (ipython-input-1722398562.py, line 2)

## TRAINING ##

In [ ]:
!pip install ultralytics


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import zipfile
import os

zip_path = "/content/pothole.v18i.yolov8.zip"
extract_path = "/content/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ تم فك الضغط في مجلد:", extract_path)


In [ ]:
for root, dirs, files in os.walk(extract_path):
    print(f"\n📁 {root}")
    for f in files:
        print(f"  - {f}")


In [ ]:
from ultralytics import YOLO


In [ ]:
model.train(
    data="/content/dataset/data.yaml",
    epochs=30,
    imgsz=640
)


# fire training

In [ ]:
!pip install ultralytics



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving fire detection.v1i.yolov8.zip to fire detection.v1i.yolov8.zip


In [ ]:

import zipfile
import os

# Replace with your actual ZIP name
zip_path = "fire detection.v1i.yolov8.zip"
extract_path = "/content/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ تم فك الضغط في مجلد:", extract_path)



✅ تم فك الضغط في مجلد: /content/dataset


In [ ]:
!ls /content/dataset



data.yaml  README.dataset.txt  README.roboflow.txt  train


In [ ]:
!cat /content/dataset/data.yaml


train: ../train/images
val: ../valid/images
test: ../test/images

nc: 1
names: ['non fire']

roboflow:
  workspace: fire-detection-tuaqc
  project: fire-detection-wuywt
  version: 1
  license: CC BY 4.0
  url: https://universe.roboflow.com/fire-detection-tuaqc/fire-detection-wuywt/dataset/1

In [ ]:
yaml_content = """
train: /content/dataset/train/images
val: /content/dataset/train/images  # ما عندك validation set، فنستخدم نفس مجلد التدريب مؤقتًا
nc: 1
names: ['non fire']
"""

with open('/content/dataset/data.yaml', 'w') as f:
    f.write(yaml_content)


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # أو استخدمي yolov8s.pt لو تبين دقة أعلى

model.train(
    data="/content/dataset/data.yaml",
    epochs=20,
    imgsz=640
)



New https://pypi.org/project/ultralytics/8.3.172 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.171 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train5, nbs=64, nms=False, opset=None, optimize=False, op

train: Scanning /content/dataset/train/labels.cache... 1000 images, 926 backgrounds, 0 corrupt: 100%|██████████| 1000/1000 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.1 ms, read: 200.0±72.7 MB/s, size: 53.9 KB)


val: Scanning /content/dataset/train/labels.cache... 1000 images, 926 backgrounds, 0 corrupt: 100%|██████████| 1000/1000 [00:00<?, ?it/s]


...

                   all       1000         74      0.484      0.365      0.434       0.37



20 epochs completed in 5.944 hours.
Optimizer stripped from runs/detect/train5/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train5/weights/best.pt, 6.2MB

Validating runs/detect/train5/weights/best.pt...
Ultralytics 8.3.171 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [04:40<00:00,  8.75s/it]


                   all       1000         74      0.485      0.365      0.434       0.37
Speed: 3.4ms preprocess, 268.6ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/train5


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e2003415390>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

#Fog

In [ ]:
!pip install ultralytics


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving fog.v1i.yolov8 (1).zip to fog.v1i.yolov8 (1).zip


In [ ]:
import zipfile
import os

# Replace with your actual ZIP name
zip_path = "fog.v1i.yolov8 (1).zip"
extract_path = "/content/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ تم فك الضغط في مجلد:", extract_path)


✅ تم فك الضغط في مجلد: /content/dataset


In [ ]:
!ls /content/dataset



data.yaml  README.dataset.txt  README.roboflow.txt  train  valid


In [ ]:
!cat /content/dataset/data.yaml


train: ../train/images
val: ../valid/images
test: ../test/images

nc: 10
names: ['-', '- annotate- and create datasets', '- collaborate with your team on computer vision projects', '- collect - organize images', '- understand and search unstructured image data', 'Roboflow is an end-to-end computer vision platform that helps you', 'This dataset was exported via roboflow.com on April 15- 2024 at 2-02 AM GMT', 'car', 'custom-dataset-yolov8-fog-density - v8 2024-03-11 7-04pm', 'undefined']

roboflow:
  workspace: bad-weather-and-multisensor-fusion
  project: fog-m3b3s
  version: 1
  license: CC BY 4.0
  url: https://universe.roboflow.com/bad-weather-and-multisensor-fusion/fog-m3b3s/dataset/1

In [ ]:
yaml_content = """
train: /content/dataset/train/images
val: /content/dataset/train/images  # ما عندك validation set، فنستخدم نفس مجلد التدريب مؤقتًا
nc: 1
names: ['non fire']
"""

with open('/content/dataset/data.yaml', 'w') as f:
    f.write(yaml_content)


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # أو استخدمي yolov8s.pt لو تبين دقة أعلى

model.train(
    data="/content/dataset/data.yaml",
    epochs=20,
    imgsz=640
)



Ultralytics 8.3.171 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train6, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True

train: Scanning /content/dataset/train/labels... 7227 images, 926 backgrounds, 6188 corrupt: 100%|██████████| 7227/7227 [00:02<00:00, 2641.63it/s]

train: /content/dataset/train/images/AM_Bing_211_png_jpg.rf.ad4e62354882742b5d18dde4f5849c63.jpg: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
train: /content/dataset/train/images/AM_Bing_217_png_jpg.rf.21416bd3b1e6a471b9798fea59718cf9.jpg: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
train: /content/dataset/train/images/AM_Bing_222_png_jpg.rf.84927d38c13ddace877e26345f638c0d.jpg: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
train: /content/dataset/train/images/AM_Bing_232_png_jpg.rf.84f06908188a51c960fe6ae1b9313182.jpg: ignoring corrupt image/label: Label class 8 exceeds dataset class count 1. Possible class labels are 0-0
train: /content/dataset/train/images/AM_Bing_242_png_jpg.rf.7fc7e57cc092878a8fcc0111012e1c09.jpg: ignoring corrupt image/label: Label class 8 exceeds dataset class count 1. Possible class 

train: New cache created: /content/dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 380.1±135.6 MB/s, size: 45.3 KB)


val: Scanning /content/dataset/train/labels.cache... 7227 images, 926 backgrounds, 6188 corrupt: 100%|██████████| 7227/7227 [00:00<?, ?it/s]

...

                   all       1039        128      0.589      0.227      0.254      0.208



20 epochs completed in 6.340 hours.
Optimizer stripped from runs/detect/train6/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train6/weights/best.pt, 6.2MB

Validating runs/detect/train6/weights/best.pt...
Ultralytics 8.3.171 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [04:07<00:00,  7.49s/it]


                   all       1039        128      0.589      0.227      0.254      0.208
Speed: 2.6ms preprocess, 227.7ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train6


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7cc4963e30d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

# land

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling n

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Segformer Landslide Detection.v4i.coco-segmentation.zip to Segformer Landslide Detection.v4i.coco-segmentation.zip


In [ ]:
import zipfile

zip_path = "Segformer Landslide Detection.v4i.coco-segmentation.zip"  # غيّر حسب اسم ملفك
extract_to = "/content/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)
